In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Esse projeto parece ser uma ótima oportunidade para explorar Processamento de Linguagem Natural (NLP) e desenvolver habilidades práticas em ciência de dados!
Plano de Trabalho para o Miniprojeto
Configuração Inicial
- Objetivo: Configurar o ambiente e se familiarizar com o desafio no Kaggle.
  - Crie um ambiente no Kaggle, Google Colab ou Jupyter Notebook local.
  - Faça o download do conjunto de dados da competição.
  - Explore a página da competição para entender os objetivos, métricas de avaliação e formato de envio.
Notebook ou Relatório
Introdução
- Breve descrição do problema e dos dados:
  - Explique o objetivo: classificar tweets como relacionados ou não a desastres.
  - Descreva os dados: número de entradas, dimensões, estrutura (exemplo: texto, rótulos, IDs).
  - Discuta o papel de NLP e a relevância desse problema no mundo real.

Análise Exploratória de Dados:
- Visualize o conjunto de dados:
  - Verifique valores nulos e distribuições de classes.
  - Explore estatísticas básicas como comprimento dos tweets, presença de links/hashtags, etc.
  - Crie gráficos (ex.: histogramas, nuvens de palavras) para insights visuais.
- Limpeza de dados:
  - Remova stopwords, caracteres especiais, e transforme o texto para lowercase.
  - Aborde valores ausentes ou inconsistências nos dados.
- Plano de análise:
  - Justifique como a limpeza e exploração guiarão o próximo passo de processamento.

Arquitetura do Modelo
- Explique sua abordagem para transformar texto em dados estruturados:
  - Escolha métodos como TF-IDF, embeddings pré-treinados (ex.: GloVe, Word2Vec).
  - Justifique a escolha com base nas necessidades do projeto.
- Descreva a arquitetura do modelo:
  - Use redes neurais sequenciais como LSTM, GRU ou Transformers.
  - Argumente por que sua arquitetura atende ao desafio de classificar textos curtos como tweets.
Resultados e Análise
- Documente os experimentos:
  - Teste diferentes modelos e hiperparâmetros.
  - Compare o impacto de técnicas como dropout, early stopping, e embeddings.
  - Visualize os resultados (matriz de confusão, precisão, recall, F1-score).
- Discuta:
  - Quais estratégias ajudaram ou não?
  - Quais melhorias poderiam ser feitas?
 Conclusão 
- Resuma as principais conclusões.
- Discuta limitações e possíveis melhorias futuras.
- Explique o que você aprendeu no processo.



In [ ]:
# Importação de bibliotecas principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# NLP e pré-processamento
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# Modelos e métricas
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

# Configuração inicial
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))


In [ ]:
# Carregar os dados do Kaggle
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Visualizar os primeiros registros
print("Treinamento:")
print(train_data.head())
print("\nTeste:")
print(test_data.head())


In [ ]:
# Informações gerais
print("Informações gerais:")
print(train_data.info())

# Distribuição das classes
sns.countplot(data=train_data, x='target', palette='viridis')
plt.title("Distribuição das Classes (0 = Não Desastre, 1 = Desastre)")
plt.show()

# Comprimento dos tweets
train_data['text_length'] = train_data['text'].apply(len)
sns.histplot(train_data['text_length'], bins=30, kde=True)
plt.title("Distribuição do Comprimento dos Tweets")
plt.show()

# Nuvem de palavras
disaster_tweets = " ".join(train_data[train_data['target'] == 1]['text'])
non_disaster_tweets = " ".join(train_data[train_data['target'] == 0]['text'])

plt.figure(figsize=(10, 5))
wordcloud_disaster = WordCloud(stopwords=stop_words, background_color='white').generate(disaster_tweets)
plt.imshow(wordcloud_disaster, interpolation='bilinear')
plt.title("Nuvem de Palavras - Tweets de Desastres")
plt.axis('off')
plt.show()


In [ ]:
# Função para limpar texto
import re

def clean_text(text):
    text = re.sub(r"http\S+", "", text)  # Remover URLs
    text = re.sub(r"[^a-zA-Z]", " ", text)  # Remover caracteres não alfabéticos
    text = text.lower()  # Transformar em lowercase
    text = " ".join([word for word in text.split() if word not in stop_words])  # Remover stopwords
    return text

# Aplicar a limpeza
train_data['cleaned_text'] = train_data['text'].apply(clean_text)
test_data['cleaned_text'] = test_data['text'].apply(clean_text)

print(train_data[['text', 'cleaned_text']].head())


In [ ]:
# Parâmetros
max_words = 10000  # Vocabulário máximo
max_len = 100  # Comprimento máximo dos tweets

# Tokenização
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_data['cleaned_text'])

X_train = tokenizer.texts_to_sequences(train_data['cleaned_text'])
X_train = pad_sequences(X_train, maxlen=max_len)

# Rótulos
y_train = train_data['target']

# Dados de teste
X_test = tokenizer.texts_to_sequences(test_data['cleaned_text'])
X_test = pad_sequences(X_test, maxlen=max_len)


In [ ]:
# Construção do modelo
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

# Compilação
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Treinamento
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

history = model.fit(
    X_train_split, y_train_split,
    validation_data=(X_val, y_val),
    batch_size=32,
    epochs=10,
    callbacks=[early_stopping]
)


In [ ]:
# Avaliar no conjunto de validação
val_preds = (model.predict(X_val) > 0.5).astype(int)
print(classification_report(y_val, val_preds))

# Fazer previsões no conjunto de teste
test_preds = (model.predict(X_test) > 0.5).astype(int)

# Preparar o arquivo para envio no Kaggle
submission = pd.DataFrame({'id': test_data['id'], 'target': test_preds.flatten()})
submission.to_csv('submission.csv', index=False)


**Conclusão Final**
Este projeto foi uma excelente introdução ao mundo do NLP e à aplicação de redes neurais para análise de texto. Com base nos resultados obtidos, sabemos que as técnicas usadas foram eficazes para a tarefa proposta, mas há espaço para melhorias. A exploração de arquiteturas mais complexas e o ajuste fino do modelo têm grande potencial para aumentar a precisão e melhorar a generalização.

